# Load and clean the dataset

In [7]:
df = pd.read_csv('fifa_dataset_l2f_test.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                2000 non-null   int64  
 1   Name                      2000 non-null   object 
 2   Age                       2000 non-null   int64  
 3   Nationality               2000 non-null   object 
 4   Overall                   2000 non-null   int64  
 5   Club                      1969 non-null   object 
 6   Value                     2000 non-null   object 
 7   Wage                      2000 non-null   object 
 8   Preferred Foot            1997 non-null   object 
 9   International Reputation  1997 non-null   float64
 10  Weak Foot                 1997 non-null   float64
 11  Skill Moves               1997 non-null   float64
 12  Work Rate                 1997 non-null   object 
 13  Body Type                 1997 non-null   object 
 14  Position

In [8]:
df.columns

Index(['Unnamed: 0', 'Name', 'Age', 'Nationality', 'Overall', 'Club', 'Value',
       'Wage', 'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Position', 'Jersey Number',
       'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight',
       'Finishing', 'HeadingAccuracy', 'Dribbling', 'Reactions', 'Vision',
       'Release Clause'],
      dtype='object')

In [9]:
df = df.drop(['Unnamed: 0', 'Name', 'Nationality', 'Club', 'Jersey Number', 'Joined', 'Loaned From'], \
        axis = 1)

Could add 'Nationality', 'Club', 'Jersey Number', as cat. features (when feature hashing is performed).

In [12]:
num_features = ['Age', 'Wage', 'Height', 'Weight', 'Release Clause']
cat_features = ['Preferred Foot', 'Body Type', 'Position']
ord_features = ['Overall', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Vision', \
              'Finishing', 'HeadingAccuracy', 'Dribbling', 'Reactions']
date_features = ['Contract Valid Until']

The procedures performed below can be performed before the pipeline is implemented.

We assume that the different work rate levels are known beforehand and can be ranked in an ordinal fashion.

In [13]:
def clean_num_features(my_string):
    my_string = my_string.replace('€', '')
    if 'K' in my_string:
        my_number = float(my_string.split('K')[0]) * 10 ** 3        
    if 'M' in my_string:
        my_number = float(my_string.split('M')[0]) * 10 **6
    if my_string == '0':
        my_number = float(my_string)
    return my_number

df['Value'] = df['Value'].apply(clean_num_features)

df['Wage'] = df['Wage'].apply(clean_num_features)

df['Release Clause'] = df[df['Release Clause'].notnull()]['Release Clause'].apply(clean_num_features)

df['Weight'] = df['Weight'].str.replace('lbs', '')

df['Height'] = df['Height'].str.split("'", expand = True).iloc[:,0].astype('float').multiply(30.48) \
.add(df['Height'].str.split("'", expand = True).iloc[:,1].astype('float').multiply(2.54))

Attack_WR = df['Work Rate'].str.split('/', expand = True).iloc[:,0]
Defense_WR = df['Work Rate'].str.split('/', expand = True).iloc[:,1].str.replace(' ', '')

my_dict = {'Low':0, 'Medium':1, 'High':2}

Attack_WR = Attack_WR.map(my_dict)
Defense_WR = Defense_WR.map(my_dict)

df['Attack_WR'] = Attack_WR

df['Defense_WR'] = Defense_WR

df = df.drop('Work Rate', axis=1)

ord_features.remove('Work Rate')
ord_features.append('Attack_WR')
ord_features.append('Defense_WR')

df[num_features]=df[num_features].astype('float')
df[cat_features]=df[cat_features].astype('category')
#df[ord_features]=df[ord_features].astype('Int64') # Should be OK to use float
df[ord_features]=df[ord_features].astype('float')

df['Value'] = df['Value'].astype('float')

skipPosition = False # Set in order to have a clean start.